# **Subset the LULCC data to shapefile**

By Bridget Bittmann

Date created: March 14, 2022

In [ ]:
## IMPORT PACKAGES ##

import numpy as np #basic computation
!pip install geopandas
import geopandas as gpd #geopandas for .shp
import matplotlib.pyplot as plt #to create plots
import pandas as pd #to create dataframes and export .csv
!pip install rasterio
import rasterio as rso #import GeoTiff files
from rasterio.mask import mask #to crop data to a boundary
from rasterio.plot import show #to plot the image
from rasterio.crs import CRS
from shapely.ops import unary_union #creates boundary of shapefile
import json #imports metadata
!pip install rioxarray #to clip rasters to a .shp file
import rioxarray as rxr
from rasterio.warp import calculate_default_transform, reproject, Resampling
!pip install pylandstats
import pylandstats #to perform landscape metrics
from pylandstats import landscape
from pylandstats import SpatioTemporalAnalysis #to calculate landscape metrics through time
import glob
import os

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
## NAVIGATE TO YOUR DIRECTORY ##
%cd gdrive/MyDrive/spatial_colab/datasets/
%ls

/content/gdrive/MyDrive/spatial_colab/datasets
2005a.tif  2010.tif  2016.tif    2021.tif               irrig_lbrb/
2006.tif   2011.tif  2017.tif    Boise_CDL/             IrrMapper/
2007a.tif  2012.tif  2018.tif    CDL_reproject/         LBRB_shp/
2007b.tif  2013.tif  2019.tif    CDL_subsets/           lcmap_files/
2008.tif   2014.tif  2020.tif    diversion_timeseries/  masked/
2009.tif   2015.tif  2021_clip/  irrigation_companies/  subset_test_shp/


In [ ]:
## CREATE A FUNCTION TO SUBSET GEOSPATIAL DATA ##
## ------------------------------------------- ##

shp_file = gpd.read_file('subset_test_shp/subset.shp') #open shapefile
names = shp_file['DIV_NAME_']
files = glob.glob('lcmap_files/*.tiff') #get all the years of cdl imagery
data =[]
for i in range(len(files)):
  data.append(rso.open(files[i])) #open cdl image and append to a list
print(data)
shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
years = np.arange(2008, 2022) #years of CDL data
collection = []
for i in range(len(shp)):
  for n in range(len(years)):
    extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
    coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
    out_img, out_transform = mask(dataset=data[n], shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
    out_meta = data[n].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform})
    # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
    in_file = files[n]
    out_filename = os.path.join('lcmap_files/masked_lcmap/'+names[i]+'_'+str(years[n])+'.tif') #create a file name to export to
    # Save clipped land cover coverage THIS WILL OVERWRITE FILES
    out_tiff = rso.open(out_filename, 'w', **out_meta)
    out_tiff.write(np.squeeze(out_img, 0), 1)
    out_tiff.close()
    collection.append(out_img)

In [17]:
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##

temporal_group = []

for i in range(len(names)):
  files = glob.glob('lcmap_files/masked_lcmap/'+names[i]+'_*.tif') #name for all the csv files
  sta = SpatioTemporalAnalysis(files, dates=years, nodata=0) #import all CDL rasters and mask
  temporal_group.append(sta)

In [19]:
# CALCULATE THE CLASS PROPORTIONS #
# ------------------------------- #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  proportions.append(df)

In [20]:
display(proportions[1])

metric           proportion_of_landscape
class_val dates                         
1         2008                  3.063479
          2009                  3.063479
          2010                  3.194054
          2011                  3.113700
          2012                  3.204098
...                                  ...
6         2017                  0.733226
          2018                  0.733226
          2019                  0.733226
          2020                  0.733226
          2021                  0.733226

[70 rows x 1 columns]

In [ ]:
## EXTRA BUT NOT READY TO DELETE ##


years = np.arange(2008, 2022)
shp_file = gpd.read_file('subset_test_shp/subset.shp') #open shapefile
data = rso.open('Boise_CDL/CDLs_boisearea/RasExt_CDL_2010_30m.tif') #open cdl image
shp_file = shp_file.to_crs(data.crs) #reproject the shp file to same projection
shp_file.crs